<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Agregaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Agregación de datos y operaciones de grupo](#scrollTo=BFynk27lvXm6)

>>[Actividades que veremos en este apartado](#scrollTo=EMGwZJuAv306)

>>>[Mecánica del GroupBy](#scrollTo=F1ju7vADwWy1)

>>>[Seleccionando una columna o subset de columnas](#scrollTo=_SAtsTx-1xjS)

>>>[Agrupando con dicts y series](#scrollTo=SFpRZZb23nOj)

>>>[Agrupación con funciones](#scrollTo=hNvhRjQg5IhT)

>>>[Data Aggregation](#scrollTo=tCCi7PrE5ecH)

>>>[Aplicación de columna inteligente y de funciones múltiples](#scrollTo=rbeK_M1blP_0)

>>[Aplicar: general dividir-aplicar-combinar](#scrollTo=K8sgtonjqmcI)

>>>[Análisis de cuantiles y buckets](#scrollTo=mFfqx7P0sbeq)

>>>[Rellenar valores perdidos con valores específicos de grupo](#scrollTo=tvKXddD9tV0C)

>>[Muestreo aleatorio y permutación](#scrollTo=vTsm2BuHv_SC)

>>>[Promedio ponderado grupal y correlación](#scrollTo=eX6B4Plhxr8P)

>>[Pivot Tables y tabulación cruzada](#scrollTo=3e5jR8qh0UMW)

>>>[Tabulaciones cruzadas (crosstab)](#scrollTo=gprrjJ0m12nf)



# Agregación de datos y operaciones de grupo

La categorización de un conjunto de datos y la aplicación de una función a cada grupo, ya sea una agregación o transformación, es un componente crítico del trabajo de análisis de datos. Después de cargar, fusionar y preparar un conjunto de datos, es posible que debas calcular estadísticas de grupo o posiblemente tablas dinámicas para fines de informes o visualización. Pandas proporciona una interfaz de grupo flexible, que te permite cortar, y resumir conjuntos de datos de forma natural.

Como verás, con la expresividad de Python y pandas, podemos realizar operaciones grupales bastante complejas utilizando cualquier función que acepte un objeto Pandas o una matriz NumPy. 

## Actividades que veremos en este apartado

* Dividir un Dataframe en pedazos usando una o más claves (en forma de funciones, matrices o nombres de columna de DataFrame).

* Calcular estadísticas de resumen de grupo, como conteo, media o desviación estándar, o una función definida por el usuario.

* Aplicar transformaciones como normalización, regresión lineal, clasificación o selección de subconjuntos.

* Calcular tablas dinámicas y tabulaciones cruzadas.

* Realizar análisis de cuantiles y otros análisis de grupos estadísticos.

### Mecánica del GroupBy

Existe un término conocido entre los analistas que describe operaciones de grupo, *split-apply-combine*.

En la primera parte de este proceso dividimos dataframes o series (split) en grupos basados en una o más keys. Una vez realizado la división, realizamos la función *apply* a cada grupo, produciendo un nuevo valor.

Finalmente, tomamos el resultado de esas operaciones y las combinamos en un objeto.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*



In [1]:
#@title
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.383160,-0.036662
1,a,two,0.070156,0.345705
2,b,one,-0.420877,-2.803723
3,b,two,0.318160,-0.002953
4,a,one,-0.136344,-1.446252


Supongamos que deseas calcular la *media* de la columna data1 usando las etiquetas de key1

In [2]:
#@title
grouped = df[['data1','data2']].groupby(df['key1']).mean()
print(grouped)

         data1     data2
key1                    
a    -0.149782 -0.379070
b    -0.051359 -1.403338


In [3]:
#@title
grouped.mean()

data1   -0.100570
data2   -0.891204
dtype: float64

Aquí agrupamos los datos usando dos claves, y la Serie resultante ahora tiene un índice jerárquico.

In [4]:
#@title
media = df['data1'].groupby([df['key1'], df['key2']]).mean()
media

key1  key2
a     one    -0.259752
      two     0.070156
b     one    -0.420877
      two     0.318160
Name: data1, dtype: float64

In [5]:
#@title
media.unstack()

key2,one,two
key1,,
a,-0.259752,0.070156
b,-0.420877,0.318160


En el siguiente ejemplo todo el grupo de keys son series

In [6]:
#@title
prov = np.array(['Buenos Aires', 'Buenos Aires', 'Córdoba', 'Córdoba', 'Tucumán'])
anios = np.array([2005, 2005, 2005, 2006, 2006])

In [7]:
df

,key1,key2,data1,data2
0,a,one,-0.383160,-0.036662
1,a,two,0.070156,0.345705
2,b,one,-0.420877,-2.803723
3,b,two,0.318160,-0.002953
4,a,one,-0.136344,-1.446252


In [8]:
#@title
print(df['data1'])
print(prov)
print(anios)

0   -0.383160
1    0.070156
2   -0.420877
3    0.318160
4   -0.136344
Name: data1, dtype: float64
['Buenos Aires' 'Buenos Aires' 'Córdoba' 'Córdoba' 'Tucumán']
[2005 2005 2005 2006 2006]


In [9]:
#@title
df['data1'].groupby([prov, anios]).mean()

Buenos Aires  2005   -0.156502
Córdoba       2005   -0.420877
              2006    0.318160
Tucumán       2006   -0.136344
Name: data1, dtype: float64

### Seleccionando una columna o subset de columnas

La indexación de un objeto **GroupBy** creado a partir de un DataFrame con un nombre de columna o matriz de nombres de columna, genera un subconjunto de columnas para la agregación.

In [10]:
df.key1

0    a
1    a
2    b
3    b
4    a
Name: key1, dtype: object

In [11]:
#@title

df.groupby('key1')['data1'].mean() # Equivalente a df['data1'].groupby(df['key1'])
df.groupby('key1')['data2'].mean() # Equivalente a df[['data2']].groupby(df['key1'])

key1
a   -0.379070
b   -1.403338
Name: data2, dtype: float64

Especialmente para grandes conjuntos de datos, puede ser conveniente agregar solo unas pocas columnas. Por ejemplo, en el conjunto de datos anterior, para calcular promedios solo para la columna data2 y obtener el resultado como un DataFrame, podríamos escribir:

In [12]:
#@title
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.259752
     two   0.070156
b    one  -0.420877
     two   0.318160

El objeto devuelto por esta operación de indexación es un DataFrame agrupado.

Será una lista o matriz o una Serie agrupada si solo se pasa un solo nombre de columna como escalar

In [13]:
#@title
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one    -0.741457
      two     0.345705
b     one    -2.803723
      two    -0.002953
Name: data2, dtype: float64

In [14]:
var=s_grouped.mean()

In [15]:
var.unstack()

key2,one,two
key1,,
a,-0.741457,0.345705
b,-2.803723,-0.002953


### Agrupando con dicts y series

Puede que necesites agrupar información existente en algo diferente a un arreglo. Consideremos el siguiente Dataframe:


In [16]:
#@title
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.415692,-0.514233,0.193808,-0.277779,-0.784613
Steve,-1.260762,0.180235,0.244247,-0.454560,-0.460955
Wes,0.238535,-0.278717,-0.238667,-0.323797,0.676341
Jim,-0.548124,0.488751,1.274600,-1.229085,0.057389
Travis,0.709180,0.404599,0.210227,0.979321,-0.744091


In [17]:
#@title
people.iloc[2:3, [1, 2]] = np.nan # Agrega un par de NaN
people

,a,b,c,d,e
Joe,0.415692,-0.514233,0.193808,-0.277779,-0.784613
Steve,-1.260762,0.180235,0.244247,-0.454560,-0.460955
Wes,0.238535,NaN,NaN,-0.323797,0.676341
Jim,-0.548124,0.488751,1.274600,-1.229085,0.057389
Travis,0.709180,0.404599,0.210227,0.979321,-0.744091


Supongamos que tenemos una lista de columnas que corresponden a ese Dataframe y queremos realizar una operacion **sum** entre las columnas por grupo

In [66]:
#@title
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

Ahora podemos construir un arreglo a partir del dict y se lo pasamos a la operación **groupby**, pero en cambio le pasamos directamente el dict como key.


In [67]:
#@title
by_column=people.groupby(mapping, axis=1).mean()
display(by_column)

,blue,red
Joe,-0.041985,-0.294385
Steve,-0.105157,-0.513827
Wes,-0.323797,0.457438
Jim,0.022758,-0.000661
Travis,0.594774,0.123229


### Agrupación con funciones
El uso de las funciones de Python es una forma más genérica de definir un mapeo de grupo en comparación con un dict o Series. 

**Cualquier función que se pase como clave de grupo se llamará una vez por valor de índice**, y los valores de retorno se utilizarán como nombres de grupo. Más concretamente, consideremos el DataFrame de ejemplo de la sección anterior, que tiene los nombres de las personas como valores de índice. Supongamos que deseas agrupar por la longitud de los nombres; Si bien podrías calcular una matriz de longitudes de cadena, es más simple simplemente pasar la función len:

In [68]:
#@title
people #Recordemos el Dataframe original

,a,b,c,d,e
Joe,0.415692,-0.514233,0.193808,-0.277779,-0.784613
Steve,-1.260762,0.180235,0.244247,-0.454560,-0.460955
Wes,0.238535,NaN,NaN,-0.323797,0.676341
Jim,-0.548124,0.488751,1.274600,-1.229085,0.057389
Travis,0.709180,0.404599,0.210227,0.979321,-0.744091


In [21]:
#@title
people.groupby(len).sum()

,a,b,c,d,e
3,0.106103,-0.025482,1.468408,-1.830661,-0.050883
5,-1.260762,0.180235,0.244247,-0.454560,-0.460955
6,0.709180,0.404599,0.210227,0.979321,-0.744091




---


### Data Aggregation
Las agregaciones se refieren a cualquier transformación de datos que produce valores escalares a partir de matrices. Los ejemplos anteriores han utilizado varios de ellos, como el cálculo de promedio, la suma, etc. 


Function name |	Description
------------- | -----------
count	| Número de valores no-NA en el grupo
sum	| Suma de valores no-NA
mean	| Media de valores no-NA 
median	| Mediana aritmética de valores no-NA
std, var	| Desviación y varianza estándar imparcial (denominador n - 1)
min, max	| Mínimo y máximo de valores no-NA
prod	| Producto de valores no-NA 
first, last	| Primer y último valores no-NA 




Puedes usar agregaciones de tu propio diseño y, además, llamar a cualquier método que también esté definido en el objeto agrupado. 

**Ejemplo**: Veamos por ejemplo un ejemplo de cálculo de cuantil sobre un dataframe agrupado.

Si bien el cuantil no se implementa explícitamente para GroupBy, es un método de la Serie y, por lo tanto, está disponible para su uso.  Internamente, GroupBy corta eficientemente la serie, llama a **quantile()** para cada pieza y luego ensambla esos resultados en el objeto de resultado:

In [69]:
#@title
df

,key1,key2,data1,data2
0,a,one,-0.383160,-0.036662
1,a,two,0.070156,0.345705
2,b,one,-0.420877,-2.803723
3,b,two,0.318160,-0.002953
4,a,one,-0.136344,-1.446252


In [71]:
#@title
grouped = df.groupby('key1')
grouped

In [74]:
#@title
grouped['data1'].quantile(0.5)

#Análisis del resultado:
#1. Vemos que el cuantil del 50% para la clave "a", que tenia 3 valores, coincide con el valor del promedio de los extremos.
#2. En el caso de la clave "b" que tenia dos valores, coincide con el promedio de ambos.


key1
a   -0.136344
b   -0.051359
Name: data1, dtype: float64

Puedes notar que algunos métodos como **describe** también funcionan, aunque no son agregaciones, estrictamente hablando

In [25]:
#@title
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.149782  0.226957 -0.383160 -0.259752 -0.136344 -0.033094   
b      2.0 -0.051359  0.522578 -0.420877 -0.236118 -0.051359  0.133401   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.070156   3.0 -0.379070  0.943774 -1.446252 -0.741457 -0.036662   
b     0.318160   2.0 -1.403338  1.980443 -2.803723 -2.103530 -1.403338   

                          
           75%       max  
key1                      
a     0.154521  0.345705  
b    -0.703146 -0.002953

### Agregación de columna inteligente y de funciones múltiples

Volvamos al conjunto de datos de propinas de ejemplos anteriores. Después de cargarlo con read_csv, agregamos una columna de porcentaje de propina tip_pct

In [75]:
#@title
prop = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/tips.csv')
prop['tip_pct'] = prop['tip'] / prop ['total_bill']

In [76]:
#@title
prop[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Como hemos visto, agregar una Serie o todas las columnas de un dataframe de datos es una cuestión de utilizar el agregado con la función deseada o llamar a un método como **mean** o **std**. 
Sin embargo, es posible que desees agregar usando una función diferente dependiendo de la columna, o múltiples funciones a la vez. 

In [96]:
#@title
grouped = prop.groupby(['day', 'smoker', 'time'])
grouped

In [97]:
#@title
grouped_pct = grouped['tip_pct']
grouped_pct

Ten en cuenta que para estadísticas descriptivas como las de la Tabla que hemos compartido al comienzo, igual que cuando hicimos el agrupamiento por función **len** se puede pasar el nombre de la función como una cadena, en este caso **mean**

In [98]:
#@title
grouped_pct.mean()

day   smoker  time  
Fri   No      Dinner    0.139622
              Lunch     0.187735
      Yes     Dinner    0.165347
              Lunch     0.188937
Sat   No      Dinner    0.158048
      Yes     Dinner    0.147906
Sun   No      Dinner    0.160113
      Yes     Dinner    0.187250
Thur  No      Dinner    0.159744
              Lunch     0.160311
      Yes     Lunch     0.163863
Name: tip_pct, dtype: float64

Una manera equivalente de escribir lo mismo es realizar una **agregación**, que agregará tantas columnas como se le indique a la funcion **.agg()**:

In [82]:
#@title
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [85]:
#@title
grouped_pct.agg(['min','max'])

min       max
day  smoker                    
Fri  No      0.120385  0.187735
     Yes     0.103555  0.263480
Sat  No      0.056797  0.291990
     Yes     0.035638  0.325733
Sun  No      0.059447  0.252672
     Yes     0.065660  0.710345
Thur No      0.072961  0.266312
     Yes     0.090014  0.241255

In [86]:
#@title
def peak_to_peak(arr):      # Función de agregación propia 
  return arr.max() - arr.min()

Si pasas una lista de funciones o nombres de funciones, obtiene un DataFrame con nombres de columnas tomados de las funciones.

In [87]:
#@title
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

¿Y si quisieramos ver también sobre cuántos datos se hace cada operación para cada key, qué agregaríamos?

In [88]:
#@title
grouped_pct.agg(['mean', 'std', peak_to_peak, 'count']) # ¿Qué falta?

mean       std  peak_to_peak  count
day  smoker                                         
Fri  No      0.151650  0.028123      0.067349      4
     Yes     0.174783  0.051293      0.159925     15
Sat  No      0.158048  0.039767      0.235193     45
     Yes     0.147906  0.061375      0.290095     42
Sun  No      0.160113  0.042347      0.193226     57
     Yes     0.187250  0.154134      0.644685     19
Thur No      0.160298  0.038774      0.193350     45
     Yes     0.163863  0.039389      0.151240     17



---
**Cambiar los nombres de columna resultantes de la agregación**:

No se necesita aceptar los nombres que GroupBy le da a las columnas. Si pasas una lista de tuplas (nombre, función), el primer elemento de cada tupla se usará como los nombres de columna de DataFrame.

In [89]:
#@title
grouped_pct.agg([('Promedio de tip%', 'mean'), ('Desvio de tip%', np.std)])

Promedio de tip%  Desvio de tip%
day  smoker                                  
Fri  No              0.151650        0.028123
     Yes             0.174783        0.051293
Sat  No              0.158048        0.039767
     Yes             0.147906        0.061375
Sun  No              0.160113        0.042347
     Yes             0.187250        0.154134
Thur No              0.160298        0.038774
     Yes             0.163863        0.039389

Con un DataFrame tienes más opciones, ya que puedes especificar una lista de funciones para aplicar a todas las columnas o diferentes funciones por columna.

Para comenzar, supongamos que deseamos calcular las mismas tres estadísticas para las columnas tip_pct y total_bill

In [90]:
prop

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [91]:
#@title
columnas = ['tip_pct', 'total_bill'] # Ahora tenemos una lista de columnas a diferencia del ejemplo anterior donde solo seleccionabamos

# una columna y a esa columna le aplicabamos varias funciones
functions = ['count', 'mean', 'max'] # A cada una de las columnas de la lista le aplicaremos entonces varias funciones

result = grouped[columnas].agg(functions) # A las dos columnas del DF le aplicamos las tres funciones

result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

Ahora, supongamos que deseamos aplicar funciones potencialmente diferentes a una o más de las columnas. Para hacer esto, pasamos un dict a *agg* que contenga una asignación de nombres de columna a cualquiera de las especificaciones de funciones enumeradas hasta ahora

In [92]:
#@title
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [95]:
prop

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [94]:
#@title
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'total_bill' : ['min', 'max']})

tip_pct                               total_bill       
                  min       max      mean       std        min    max
day  smoker                                                          
Fri  No      0.120385  0.187735  0.151650  0.028123      12.46  22.75
     Yes     0.103555  0.263480  0.174783  0.051293       5.75  40.17
Sat  No      0.056797  0.291990  0.158048  0.039767       7.25  48.33
     Yes     0.035638  0.325733  0.147906  0.061375       3.07  50.81
Sun  No      0.059447  0.252672  0.160113  0.042347       8.77  48.17
     Yes     0.065660  0.710345  0.187250  0.154134       7.25  45.35
Thur No      0.072961  0.266312  0.160298  0.038774       7.51  41.19
     Yes     0.090014  0.241255  0.163863  0.039389      10.34  43.11


---

## Apply

El método mas general de uso de GroupBy es **apply**.

Como se ilustra en la Figura, **apply** divide el objeto que se está manipulando en piezas, invoca la función pasada en cada pieza y luego intenta concatenar las piezas juntas.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*

Supongamos que deseamos seleccionar los cinco valores principales de **tip_pct** por grupo. Primero, escribimos una función que seleccione las filas con los valores más grandes en una columna particular:

In [40]:
#@title
# Volvamos a trabajar con el dataframe original
prop.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [41]:
#@title
def top(df, n=5, column='tip_pct'):
  return df.sort_values(by=column)[-n:] # Está haciendo un sort por columna "tip_pct" y retornando las últimas "n" filas

In [42]:
#@title
top(prop, n=6) # Llamada a la función top y reemplaza n=5 de la funcion por n=6

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Ahora, si agrupamos por fumador, por ejemplo, y llamamos a esta función, obtenemos lo siguiente:

In [43]:
#@title
prop.groupby('smoker').apply(top) # apply llama a la función top

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

¿Qué ha pasado aquí? 

La función superior se llama en cada grupo de filas desde cada split del dataframe (el primer grupo es la agrupación smoker "Yes" y el segundo es la agrupación smoker "No") y luego los resultados se pegan usando *pandas.concat*, etiquetando las piezas con los nombres de los grupos. 

Por lo tanto, el resultado tiene un índice jerárquico cuyo nivel interno contiene valores de índice del DataFrame original.

Si pasas una función a *apply* que toma otros argumentos o palabras clave, puedes pasarlos después de la función:

In [44]:
#@title
prop.groupby(['smoker', 'day']).apply(top, n=2, column='total_bill') # En este caso aplicamos sobre un agrupamiento de dos claves.

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982



---


### Análisis de cuantiles y buckets

Pandas tiene algunas herramientas, en particular *cut* y *qcut*, para dividir los datos en cubos con contenedores de tu elección o por cuantiles de muestra. La combinación de estas funciones con *groupby* hace que sea conveniente realizar análisis de buckets o cuantil en un conjunto de datos. Considere un conjunto de datos aleatorio simple y una categorización de bucket de igual longitud usando cut:

In [45]:
#@title
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame

,data1,data2
0,-0.756872,1.955538
1,0.975750,-0.340726
2,0.002472,0.951543
3,0.079874,0.993616
4,-1.187710,-0.981390
...,...,...
995,-0.635312,0.269881
996,0.577948,2.381195
997,-1.207588,1.467593
998,0.653601,-0.874000


In [46]:
#@title
quartiles = pd.cut(frame.data1, 4) # Cortamos los datos en 4 conjuntos

In [47]:
#@title
quartiles[:10]

0    (-1.475, 0.111]
1     (0.111, 1.697]
2    (-1.475, 0.111]
3    (-1.475, 0.111]
4    (-1.475, 0.111]
5     (0.111, 1.697]
6    (-1.475, 0.111]
7     (0.111, 1.697]
8     (0.111, 1.697]
9     (0.111, 1.697]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.067, -1.475] < (-1.475, 0.111] < (0.111, 1.697] <
                                           (1.697, 3.282]]

El objeto  devuelto por *cut* se puede pasar directamente a *groupby*. Entonces podríamos calcular un conjunto de estadísticas para la columna data2 de la siguiente manera:

In [48]:
#@title
def get_stats(group):
  return {'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

In [49]:
#@title
grouped = frame['data2'].groupby(quartiles)

In [50]:
#@title
grouped.apply(get_stats) # que puedo agregar para que se vea mejor? ..un___..?

data1                  
(-3.067, -1.475]  min       -1.530919
                  max        2.362553
                  count     66.000000
                  mean       0.065127
(-1.475, 0.111]   min       -2.660275
                  max        3.013319
                  count    474.000000
                  mean      -0.022622
(0.111, 1.697]    min       -2.975671
                  max        2.490842
                  count    408.000000
                  mean       0.034505
(1.697, 3.282]    min       -2.654109
                  max        2.395455
                  count     52.000000
                  mean       0.009276
Name: data2, dtype: float64

In [51]:
#@title
# Haz tu magia
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.067, -1.475]",-1.530919,2.362553,66.0,0.065127
"(-1.475, 0.111]",-2.660275,3.013319,474.0,-0.022622
"(0.111, 1.697]",-2.975671,2.490842,408.0,0.034505
"(1.697, 3.282]",-2.654109,2.395455,52.0,0.009276


### Rellenar valores perdidos con valores específicos de grupo

En clases anteriores vimos que a veces simplemente haremos **dropna** pero otras veces necesitaremos reemplazar los datos faltantes (nulos) por valores convenientes.

*fillna* es la herramienta adecuada para usar; por ejemplo, aquí rellenamos los valores de NA con la media, como vimos previamente:

In [52]:
#@title
s = pd.Series(np.random.randn(6))

In [53]:
#@title
s[:3] = np.nan
s

0         NaN
1         NaN
2         NaN
3    0.155088
4    2.276471
5    0.251322
dtype: float64

In [54]:
#@title
s.fillna(s.mean())

0    0.894294
1    0.894294
2    0.894294
3    0.155088
4    2.276471
5    0.251322
dtype: float64

**Supongamos que necesitas que el valor de relleno varíe según el grupo.**

Una forma de hacer esto es agrupar los datos y usar *apply* con una función que llame a *fillna* en cada fragmento de datos. 

Aquí hay algunos datos de muestra sobre los estados de EE. UU. Divididos en regiones orientales y occidentales:

In [55]:
#@title
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [56]:
#@title
# group_key = ['East'] * 4 + ['West'] * 4 # Notación alternativa
group_key = ['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

Ten en cuenta que la sintaxis ['Este'] * 4 produce una lista que contiene cuatro copias de los elementos en ['Este'].

In [57]:
#@title
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.043951
New York      0.852929
Vermont       0.911224
Florida       0.035381
Oregon        0.242140
Nevada       -2.006736
California   -0.080685
Idaho        -0.479776
dtype: float64

In [58]:
#@title
data['Vermont', 'Nevada', 'Idaho'] = np.nan
data

IndexingError: ignored

In [ ]:
#@title
data.groupby(group_key).mean() # Al hacer el promedio, NO contempla los nulos, no los suma.

In [ ]:
#@title
fill_mean = lambda g: g.fillna(g.mean()) # Que hace esta funcion lambda?

In [ ]:
#@title
data.groupby(group_key).apply(fill_mean)


---

## Muestreo aleatorio y permutación
Supongamos que deseas extraer una muestra aleatoria de un gran conjunto de datos para fines de simulación o alguna otra aplicación. Hay varias formas de realizar los "sorteos"; Aquí usamos el método de muestra para Series.

In [ ]:
#@title
suits = ['H', 'S', 'C', 'D'] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names) # Para cada letra itera por la cantidad de cartas

deck = pd.Series(card_val, index=cards)

Así que ahora tenemos una Serie de longitud 52 cuyo índice contiene nombres y valores de cartas que se usan en Blackjack y otros juegos (para simplificar las cosas, solo dejo que el as 'A' sea 1):


In [ ]:
#@title
deck[:13]

In [ ]:
#@title
def draw(deck, n=5): # Esta funcion recibe la serie "Deck" (el mazo) y devuelve una muestra random de "n" elementos
  return deck.sample(n)

In [ ]:
#@title
draw(deck)

Supongamos que quieres dos cartas al azar de cada palo. Debido a que el palo es el último caracter de cada nombre de tarjeta, podemos agruparlo en base a esto y usar apply:

In [ ]:
#@title
get_suit = lambda card: card[-1] # Tomo la última letra que es el palo

In [ ]:
#@title
a = deck.groupby(get_suit)
a.apply(draw, n=2)



---


### Un ejemplito de transformación y correlación entre columnas

Más adelante vamos a ver más a fondo el tema de correlación, cuando entremos a algoritmos de machine learning, pero veamos ahora una simple transformación.

Consideremos un conjunto de datos financieros originalmente obtenido de Yahoo! Finance que contiene precios al final del día para algunas acciones y el índice S&P 500 (el símbolo SPX):

In [ ]:
#@title
close_px = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/stocks.csv', 
                       parse_dates=True, index_col=0)
close_px.info()

In [ ]:
#@title
close_px

Una tarea de interés podría ser calcular un DataFrame que consta de las correlaciones anuales de los rendimientos diarios con SPX. 

1° paso: Vamos a hacer una **transformación**:


In [ ]:
#@title
rets = close_px.pct_change().dropna() #Calculamos el procentaje de cambio y eliminamos nulos
# Por defecto, la función pct_change, calcula el porcentaje de cambio entre el valor actual y el de la row inmediata anterior

Vemos que en lugar de los valores originales, ahora tenemos el porcentaje de cambio

¿Qué dato desapareció? ¿Tiene sentido?

In [ ]:
#@title
rets

2° paso: Creamos una función que calcula la correlación por pares de cada columna con la columna 'SPX':

In [ ]:
#@title
spx_corr = lambda x: x.corrwith(x['SPX']) #Esta funcion aplica a Dataframes exclusivamente y mide la correlación entre cada columna con la que se pasa como parámetro

In [ ]:
#@title
get_year = lambda x: x.year
by_year = rets.groupby(get_year) # Agrupamos los porcentajes de cambio por año
resultado = by_year.apply(spx_corr)
'''
Llama a la funcion spx_corr para calcular la correlación de cada columna del dataframe contra
la columna 'SPX', luego de hacer la agrupacion por año.
'''
resultado

Si quisieramos ver la pinta que tiene una matriz de correlación completa para todas las columnas para alguno de los años, por ejemplo para 2003 (primer fila), podemos hacer:

In [ ]:
#@title
resultado = resultado.loc[2003:2010,:]#Hago el loc que devuelve una serie y la transformo en dataframe
# Agrego "transpose" para obtener las filas como columnas
resultado

In [ ]:
#@title
resultado.corr() #Ahora si puedo correlacionar sobre el dataframe



---


## Pivot Tables y tabulación cruzada

Una tabla dinámica es una herramienta de resumen de datos que se encuentra con frecuencia en programas de hojas de cálculo. 

Agrega una tabla de datos por una o más claves, organizando los datos en un rectángulo con algunas de las claves de grupo a lo largo de las filas y algunas a lo largo de las columnas. 

Las tablas dinámicas en Python con Pandas son posibles a través de la función *groupby*. DataFrame tiene un método *pivot_table* y también hay una función *pandas.pivot_table* de nivel superior. Además de proporcionar una interfaz conveniente para *groupby*, *pivot_table* puede agregar totales parciales, también conocidos como márgenes.

Volviendo al conjunto de datos de propinas, supongamos que deseamos calcular una tabla de promedios grupales:

In [ ]:
#@title
prop.head()

In [ ]:
#@title
prop.pivot_table(index=['day', 'smoker'])
# En este caso estamos generando una agrupación con promedios por columna, 
# y lo que obtenemos es un dataframe con índices jerárquicos

Ahora, supongamos que queremos agregar solo *tip_pct* y *size*, y además agrupar por tiempo. 

Pondremos fumador en las columnas de la tabla y día en las filas:

In [ ]:
#@title
prop.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

In [ ]:
#@title
prop.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0) # Si hay NaN podemos usar fill_value


### Tabulaciones cruzadas (crosstab)
Una tabulación cruzada es un caso especial de una tabla dinámica que **calcula las frecuencias de grupo**. Aquí hay un ejemplo:

In [ ]:
#@title
pd.crosstab([prop.time, prop.day], prop.smoker)

Podríamos aumentar esta tabla para incluir totales parciales pasando 'margins=True'. Esto tiene el efecto de agregar todas las etiquetas de fila y columna, siendo los valores correspondientes las estadísticas de grupo para todos los datos dentro de un solo nivel

In [ ]:
#@title
df_cross = pd.crosstab([prop.time, prop.day], prop.smoker, margins=True)
df_cross

In [ ]:
#@title
#Si se quiere acceder a un elemento dentro de un indice jerarquico, 
# refinamos por columna y luego por indice con la jerarquia
df_cross = df_cross[['All']].loc['Dinner','Fri']
df_cross